In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import pyautogui as pg
import cv2

from PIL import Image
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [9]:
n_points = 10
page_size = pg.size()
positions = zip(np.random.randint(0, page_size[0], n_points), 
             np.random.randint(0, page_size[1], n_points))
positions = [x for x in positions]
# record_button_pos = (1635, 527)
record_button_pos = (727, 543)
positions.insert(0, record_button_pos)

position_interp = [record_button_pos]
fps = 12.0
for idx, position in enumerate(positions):
    if idx == n_points:
        break
    else:
        steps = int(fps)
        split_x = (positions[idx+1][0] - positions[idx][0]) // steps
        split_y = (positions[idx+1][1] - positions[idx][1]) // steps
        for each in np.arange(1, steps):
            position_interp.append((position[0] + split_x*each, position[1] + split_y*each))
        position_interp.append(positions[idx+1])
times = []

# Create VideoCapture object
cap = cv2.VideoCapture(0)

for i in np.arange(len(position_interp)-1):
    # Move the mouse to new position
    pg.moveTo(position_interp[i+1][0], position_interp[i+1][1], duration=0.15)
    
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    cv2.imwrite('training_images\\test' + str(i) + '.png', gray)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
# out.release()
cv2.destroyAllWindows()
pickle.dump(position_interp, open('position_data.p', 'wb'))

In [40]:
cap.release()
# out.release()
cv2.destroyAllWindows()

In [2]:
import tensorflow as tf
from PIL import Image

C:\toolkits.win\anaconda3-4.4.0\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=307200, kernel_initializer='normal', activation='relu'))
    model.add(Dense(2, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [12]:
position_interp = pickle.load(open('position_data.p', 'rb'))
y = np.array(np.array(position_interp) - 1)
X = np.zeros((y.shape[0], 307200))

for i in range(len(y)-1):
    # Read in the new image
    img = Image.open('training_images/test' + str(i) + '.png')
    
    # Transform image into 1-D array
    arr = np.array(img)
    shape = arr.shape
    flat_arr = arr.ravel()
    
    # Append flattened array to X
    X[i] = flat_arr


In [87]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=5, verbose=0)

In [88]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

InternalError: Failed to create session.

In [58]:
blah = [(2, 5), (2, 3)]
print(np.array(np.array(blah)))
print(type(np.array(blah)))

[[2 5]
 [2 3]]
<class 'numpy.ndarray'>
